In [212]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Font
from openpyxl.comments import Comment
from utilities import add_comment

In [226]:
old_path =  'Test_Data/note_eleve.xlsx'
new_path = 'Test_Data/new_note.xlsx'
output = 'Test_Data/output.xlsx'

In [227]:
df_old = pd.read_excel(old_path)
df_new = pd.read_excel(new_path)

In [228]:
df_old,df_new.dtypes

(      Nom     prenom  note
 0   nom_1   prénom_1  19.0
 1   nom_2   prénom_2   6.0
 2   nom_3   prénom_3  17.0
 3   nom_4   prénom_4   NaN
 4   nom_5   prénom_5  10.0
 5   nom_6   prénom_6  11.0
 6   nom_7   prénom_7  15.0
 7   nom_8   prénom_8  13.0
 8   nom_9   prénom_9   9.0
 9  nom_11  prénom_10  10.0,
 Nom        object
 prenom     object
 note      float64
 dtype: object)

In [229]:
len(list(df_old.columns)) - len(list(df_new.columns))

0

In [230]:
columns = list(df_old.columns)

In [231]:
print(f'Shape of new data : {df_new.shape}')
print(f'Shape of ref data : {df_old.shape}')

Shape of new data : (11, 3)
Shape of ref data : (10, 3)


In [233]:
uniques_id = np.random.randint(1, len(df_old)*100, size=len(df_old))

if df_new.shape[0] > df_old.shape[1]:
    df_old['_id'] = uniques_id
    df_new['_id'] = np.concatenate((uniques_id, [np.nan] * max(0, len(df_new) - len(uniques_id))))
    
elif df_new.shape[0] < df_old.shape[1]:
    df_old['_id'] = uniques_id
    df_new['_id'] = uniques_id[:len(df_new)]
else : 
    df_old['_id'] = uniques_id
    df_new['_id'] = uniques_id

In [221]:
Deleted_Rows = pd.merge(df_old, df_new, on='_id', how ='left', indicator = True)

In [222]:
Deleted_Rows = Deleted_Rows[Deleted_Rows['_merge'] == 'left_only'].drop(columns = '_merge')

In [223]:
Deleted_Rows

,Nom_x,prenom_x,note_x,_id,Nom_y,prenom_y,note_y


In [15]:
red_fill = PatternFill(start_color='FF0000', end_color='FF0000', fill_type='solid')
font = Font(color='FFFFFF')

In [16]:
Deleted_Rows['id'].values

array([5, 9])

In [24]:
Deleted_Rows[Deleted_Rows['id']==5].index[0]

np.int64(4)

In [46]:
wb_old = load_workbook(old_path)
ws_old = wb_old.active

idx_Deleted_rows = Deleted_Rows['id'].values
for idx_item in idx_Deleted_rows :
    for col in range(1, ws_old.max_column + 1):
        idx_row = Deleted_Rows[Deleted_Rows['id']==idx_item].index[0]
        cell = ws_old.cell(row=idx_row+2, column=col)
        cell.fill = red_fill
    ws_old.cell(row=idx_row+2, column=1).comment = Comment("Deleted Row", "Author")
wb_old.save(output)
print(f'Removed rows have been highlighted in red in {output}')

Removed rows have been highlighted in red in Test_Data/output.xlsx


In [40]:
Unchanged_Rows = pd.merge(df_old, df_new, on=columns[:-1], how = 'inner', indicator = True)

In [41]:
Unchanged_Rows = Unchanged_Rows[Unchanged_Rows['_merge'] == 'both'].drop(columns = '_merge')
Unchanged_Rows

,id,Nom,prenom,note_x,note_y
0,1,nom_1,prénom_1,19.0,19.0
1,3,nom_3,prénom_3,17.0,17.0
2,7,nom_7,prénom_7,15.0,15.0
3,8,nom_8,prénom_8,13.0,13.0


In [42]:
red_fill = PatternFill(start_color='808080', end_color='808080', fill_type='solid')
font = Font(color='FFFFFF')

In [43]:
Unchanged_Rows['id'].values

array([1, 3, 7, 8])

In [45]:
# wb_old = load_workbook(old_path)
# ws_old = wb_old.active

idx_Unchanged_Rows = Unchanged_Rows['id'].values
for idx_item in idx_Unchanged_Rows :
    for col in range(1, ws_old.max_column + 1):
        idx_row = Unchanged_Rows[Unchanged_Rows['id']==idx_item].index[0]
        cell = ws_old.cell(row=idx_row+2, column=col)
        cell.fill = red_fill
    ws_old.cell(row=idx_row+2, column=1).comment = Comment("Deleted Row", "Author")
print(f'Removed rows have been highlighted in red in {output}')

Removed rows have been highlighted in red in Test_Data/output.xlsx


In [186]:
if df_old.shape == df_new.shape :
    modified_cells = df_old.compare(df_new)
    
    with pd.ExcelWriter(output, engine='openpyxl') as writer:
        
        modified_cells.to_excel(writer, sheet_name='Modified')
    
    df = pd.read_excel(output, header=[0, 1]) 
    
    wb = load_workbook(output)
    ws = wb.active
    
    red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")  # Rouge
    gray_fill = PatternFill(start_color="D3D3D3", end_color="D3D3D3", fill_type="solid")  # Gris
    
    self_columns = [col for col in df.columns if col[1] == "self"]
    other_columns = [col for col in df.columns if col[1] == "other"]
    
    for row in range(2, len(df) + 2): 
        for self_col, other_col in zip(self_columns, other_columns):
            self_value = df.at[row - 2, self_col]
            other_value = df.at[row - 2, other_col]
    
            cell_self = ws.cell(row=row + 1, column=df.columns.get_loc(self_col) + 1)
            cell_other = ws.cell(row=row + 1, column=df.columns.get_loc(other_col) + 1)
    
            if pd.notna(self_value) and pd.notna(other_value) and self_value != other_value:
                cell_self.fill = red_fill
                cell_other.fill = red_fill
    
            if pd.isna(self_value):
                cell_self.fill = gray_fill
            if pd.isna(other_value):
                cell_other.fill = gray_fill
    
    wb.save(output)
    wb.close()
    print("Comparison complete. Check 'comparison_result.xlsx' for details.")
else :
    with pd.ExcelWriter(output, engine="openpyxl", mode="w") as writer:
        df_final.to_excel(writer, sheet_name="Added Row", index=False)
        df_final.to_excel(writer, sheet_name="Removed Row", index=False)
        df_final.to_excel(writer, sheet_name="Modified Row", index=False)